## Домашняя работа

1. Наборы данных вида Transactions (несколько транзакций на одного клиента) трансформировать в таблицу, где cl_id будут уникальными (соответственно 4000 строк в train и 1000 строк в test
2. Для каждого cl_id будет уникальное целевое событие target_flag, а также уникальный канал привлечения клиента channel_type (клиент привлекается лишь однажды и с самого начала его записи присваивается значение канала привлечения)
3. При агрегации (*pandas.DataFrame.groupby*) по cl_id (или по связке cl_id, channel_type, target_flag) необходимо создавать производные фичи, идеи для таких фичей могут быть следующими:

    - общая сумма транзакций по каждой из trx_category
    - общая сумма транзакции по основным вылютам (напр. выделить рубли, доллары и евро - предположительно, это будут самые крупные категории)
    - общая сумма транзакций по категориям MCC кодов (например, выбрать основные/популярные MCC коды). ВНИМАНИ! Некоторые MCC коды из train могут быть не представлены в test. Про MCC коды в целом: http://www.banki.ru/wikibank/mcc-kod/; Справочник MCC кодов: https://mcc-codes.ru/code; Про некоторые категории кэшбека Росбанка: https://mcc-codes.ru/card/rosbank-sverkh-plus;
    - возможные агрегации по времени суток и дням недели - траты в выходные (праздники) или будни, в ночное время или в рабочее и т.д.
4. **Обязательная часть**: провести первичный анализ данных - посмотреть распределения признаков, выделить самые популярные MCC, помотреть активность клиентов по дням недели/времени, какие категории транзакции (trx_category) наиболее популярны и т.д. Получить инсайты, которые в дальнейшем помогут вам правильно подготовить фичи
5. **Дополнительная часть**: с отобранными фичами и полученными компонентами обучить модель (тип алгоритма на свой вкус, можно начать с линейной) и померить качество на локальном тестовом наборе данных (локальная валидация), который создается в этом ноутбуке. **Метрика оценки качества - ROC AUC**(https://en.wikipedia.org/wiki/Receiver_operating_characteristic)
6. Задания принимаются в виде ноутбука с кодов/картинками выполненной обязательной части


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime 
from datetime import timedelta
import sqlite3
import seaborn as sns

In [5]:
raw_df = pd.read_csv('rosbank_train.csv')
raw_df.head(5)

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum
0,01/10/2017,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,0,0.0
1,01/10/2017,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,0,0.0
2,01/12/2017,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,0,0.0
3,01/10/2017,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,0,0.0
4,01/10/2017,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,0,0.0


In [6]:
conn = sqlite3.connect('transactions.db')

In [7]:
datamart_check = pd.io.sql.read_sql("""
    select
        cl_id,
        target_flag,
        channel_type,
        currency,
        sum(case when MCC = 5411 then amount else 0 end) as MCC_5411,
        sum(case when MCC = 6011 then amount else 0 end) as MCC_6011,
        sum(case when MCC = 5814 then amount else 0 end) as MCC_5814,
        sum(case when MCC = 5812 then amount else 0 end) as MCC_5812,
        sum(case when MCC = 5499 then amount else 0 end) as MCC_5499
    from transactions_raw
    group by 1, 2
    order by 1
    """, conn)

#уникальное целевое событие target_flag по клиенту, а также уникальный канал привлечения клиента channel_type добавила сразу при агрегации

In [8]:
#я тут очень страшное навертела, полагаю, что можно было бы проще, буду очень рада, если вы поможете понять как это страшное оптимизировать
datamart_check['index'] = datamart_check.index
MCC_avg1 = (datamart_check['MCC_5411'] + datamart_check['MCC_6011'] + datamart_check['MCC_5814'] + datamart_check['MCC_5814'] + datamart_check['MCC_5812'] + datamart_check['MCC_5499'])
MCC_avg2 = pd.DataFrame(MCC_avg1, columns=['MCC_avg'])
MCC_avg2['index'] = MCC_avg2.index
datamart_check1 = pd.merge(datamart_check, MCC_avg2, on='index', how='left')
del datamart_check1['index']
datamart_check1.head()

#решила общую сумму транзакций по категориям MCC кодов допилить отдельной колонкой до разбиения таблицы на train/test

,cl_id,target_flag,channel_type,currency,MCC_5411,MCC_6011,MCC_5814,MCC_5812,MCC_5499,MCC_avg
0,0,0,None,810,2031.00,20000.0,0.00,0.00,0.00,22031.00
1,1,0,None,810,14629.46,145000.0,82441.17,3180.01,492.80,328184.61
2,5,1,None,810,44925.40,208000.0,247.00,3309.00,7196.70,263925.10
3,9,0,None,810,39.00,540000.0,0.00,0.00,0.00,540039.00
4,10,0,None,810,53920.36,854000.0,2010.00,319.00,13225.02,925484.38


In [71]:
print("Total clients: ", len(datamart_check1.cl_id.unique()))

Total clients:  5000


In [9]:
#test_data
cl_ids_test = datamart_check1.sample(1000)
cl_ids_test.head()


,cl_id,target_flag,channel_type,currency,MCC_5411,MCC_6011,MCC_5814,MCC_5812,MCC_5499,MCC_avg
1435,2897,0,type2,810,27901.00,906400.0,4522.5,0.0,0.00,943346.00
976,2007,1,type2,810,53351.92,320000.0,0.0,0.0,0.00,373351.92
794,1613,1,type2,810,8420.54,168000.0,4867.4,0.0,14085.09,200240.43
3679,7571,0,type1,810,10612.85,88000.0,86.0,2363.0,113.29,101261.14
2477,5030,1,type1,810,15316.00,42000.0,0.0,0.0,0.00,57316.00


In [10]:
#train_data
cl_ids_train = datamart_check1.sample(4000)
cl_ids_train.head()

,cl_id,target_flag,channel_type,currency,MCC_5411,MCC_6011,MCC_5814,MCC_5812,MCC_5499,MCC_avg
3426,7023,0,type1,810,39685.08,65000.0,0.0,0.00,0.0,104685.08
3040,6201,0,type1,810,24977.81,128000.0,2258.0,0.00,0.0,157493.81
1815,3681,1,type1,810,59352.28,0.0,34741.0,13359.25,952.0,143145.53
3383,6938,0,type1,810,65839.13,221500.0,0.0,3973.50,5310.0,296622.63
666,1328,1,type2,810,6077.85,75200.0,0.0,850.00,5929.0,88056.85


In [11]:
#смотрим клиентов, у которых не было ни одной транзакции по указанным MCC, их 100 
datamart_check1 = cl_ids_train[cl_ids_train['MCC_avg'] == 0.0].sort_values('cl_id', ascending=False)
datamart_check1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 4991 to 34
Data columns (total 10 columns):
cl_id           100 non-null int64
target_flag     100 non-null int64
channel_type    100 non-null object
currency        100 non-null int64
MCC_5411        100 non-null float64
MCC_6011        100 non-null float64
MCC_5814        100 non-null float64
MCC_5812        100 non-null float64
MCC_5499        100 non-null float64
MCC_avg         100 non-null float64
dtypes: float64(6), int64(3), object(1)
memory usage: 8.6+ KB


In [12]:
raw_df.head(5)

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum
0,01/10/2017,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,0,0.0
1,01/10/2017,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,0,0.0
2,01/12/2017,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,0,0.0
3,01/10/2017,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,0,0.0
4,01/10/2017,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,0,0.0


In [13]:
#список самых популярных MCC, использую raw_df, вроде в условии не прописано, что по train_data надо работать? 
raw_df['MCC'].value_counts().head()

5411    121640
6011     54382
5814     41351
5812     30027
5499     27237
Name: MCC, dtype: int64

In [14]:
#смотрим по какой из катерий чаще всего проходят транзакции
raw_df['trx_category'].value_counts()

POS               416425
DEPOSIT            21216
WD_ATM_ROS         19104
WD_ATM_PARTNER      9948
C2C_IN              7306
WD_ATM_OTHER        7140
C2C_OUT             5456
BACK_TRX            2687
CAT                 1197
CASH_ADV              34
Name: trx_category, dtype: int64

In [15]:
#сумма всех транзакций по trx_category
a = raw_df.groupby('trx_category').sum().sort_values('amount', ascending=True)
a['amount'].head()

trx_category
CASH_ADV          912824.94
CAT             22602859.61
BACK_TRX        24619173.63
WD_ATM_OTHER    74780620.98
C2C_OUT         88074878.79
Name: amount, dtype: float64

In [ ]:
#не получилось ничего посортировать по датам, не понимаю как в таблице менять и как сортировать по ним :((